# SRT Subtitle Interpreter
## A Complete Language Processing System

**Course:** CSS125L - Programming Languages  
**Project:** Machine Project - Interpreter Design  
**Language:** Python 3.13+

---

# Section 1: Introduction

## What is an Interpreter?

An **interpreter** is a program that directly executes instructions written in a programming or scripting language without requiring them to be compiled into machine code first. Unlike compilers that translate source code to executable binaries, interpreters process and execute code line-by-line or statement-by-statement.

**Compiler vs Interpreter:**
- **Compiler**: Source Code → Compilation → Machine Code → Execution
- **Interpreter**: Source Code → Direct Execution (with optional parsing/translation steps)

## Why Subtitles Need Interpreters

Subtitle files (.srt, .vtt, .ass) contain:
1. **Timing information** - When each subtitle should appear and disappear
2. **Text content** - The actual subtitle text
3. **Formatting metadata** - Styling, positioning, colors

An interpreter is needed to:
- **Parse** the subtitle format (lexical and syntax analysis)
- **Validate** timing constraints (no overlaps, sequential ordering)
- **Execute** time-synchronized display
- **Transform** content (translation, formatting, export)

## Real-World Applications

SRT subtitle processing is used in:
- **Video Players** (VLC, MPC-HC) - Real-time subtitle rendering
- **Streaming Services** (Netflix, YouTube) - Multi-language subtitle management
- **Subtitle Editors** (Aegisub, Subtitle Edit) - Creation and modification
- **Translation Services** - Automated subtitle localization
- **Accessibility Tools** - Closed captioning for hearing impaired

## Our Interpreter System

This project implements a complete SRT subtitle interpreter with:

**Core Features:**
- Lexical analysis (tokenization)
- Syntax parsing with validation
- Abstract Syntax Tree (AST) construction
- Time-synchronized execution (3 modes)

**Advanced Features:**
- Multi-language translation (5 languages)
- Statistics calculation
- Export functionality (text and SRT)
- ANSI formatting for HTML tags

**Design Goals:**
- Educational clarity over optimization
- Comprehensive error handling
- Modular, testable architecture
- Real-world applicability

# Section 2: Input Language Description

## SRT Format Specification

The SubRip Text (.srt) format is a simple subtitle format with the following structure:

```
<index>
<start_timestamp> --> <end_timestamp>
<subtitle_text_line_1>
[<subtitle_text_line_2>...]
<blank_line>
```

**Components:**
1. **Index**: Sequential number (1, 2, 3, ...)
2. **Timestamp line**: `HH:MM:SS,mmm --> HH:MM:SS,mmm`
   - Hours: 00-99
   - Minutes: 00-59
   - Seconds: 00-59
   - Milliseconds: 000-999
3. **Text lines**: One or more lines of subtitle text
4. **Blank line**: Separator between subtitle blocks

**Optional Features:**
- HTML-like formatting tags: `<i>`, `<b>`, `<u>`, `<font color="#RRGGBB">`
- Multi-line text content
- UTF-8 encoding for international characters

In [ ]:
# Example: Valid SRT File
with open('examples/valid_basic.srt', 'r', encoding='utf-8') as f:
    content = f.read()
    
print("=" * 60)
print("VALID SRT FILE: examples/valid_basic.srt")
print("=" * 60)
print(content)

In [ ]:
# Example: Invalid SRT Files
invalid_files = [
    ('examples/invalid_missing_index.srt', 'Missing index number'),
    ('examples/invalid_timestamp_order.srt', 'Start time after end time')
]

for filepath, description in invalid_files:
    print("=" * 60)
    print(f"INVALID: {filepath}")
    print(f"Error: {description}")
    print("=" * 60)
    with open(filepath, 'r', encoding='utf-8') as f:
        print(f.read())
    print()

## Token Types and Grammar Rules

### Token Types

Our lexer recognizes the following token types:

1. **INDEX**: `^\d+$` - Sequential subtitle number
2. **TIMESTAMP**: `\d{2}:\d{2}:\d{2},\d{3}` - Time in HH:MM:SS,mmm format
3. **ARROW**: `-->` - Separator between start and end timestamps
4. **TEXT**: Any non-empty line that isn't index/timestamp/arrow
5. **FORMATTING_TAG**: HTML-like tags (`<i>`, `</i>`, `<b>`, etc.)
6. **NEWLINE**: `\n` - Line break
7. **BLANK_LINE**: `\n\n` - Subtitle block separator
8. **EOF**: End of file marker

### Formal Grammar (EBNF)

```ebnf
subtitle_file    = subtitle_block+ EOF
subtitle_block   = INDEX NEWLINE timestamp_line NEWLINE text_lines BLANK_LINE
timestamp_line   = TIMESTAMP ARROW TIMESTAMP
text_lines       = TEXT (NEWLINE TEXT)*
```

### Validation Rules

1. **Sequential indexes**: Must be 1, 2, 3, ... (no gaps)
2. **Timestamp validity**: MM ≤ 59, SS ≤ 59, mmm ≤ 999
3. **Time ordering**: start_time < end_time for each subtitle
4. **Non-empty text**: At least one text line required
5. **Block structure**: Proper blank line separation

## Design Rationale

**Why .srt is ideal for an interpreter project:**

1. **Clear lexical structure** - Easy to tokenize with regex patterns
2. **Simple grammar** - Straightforward parsing rules
3. **Rich validation opportunities** - Temporal, sequential, structural constraints
4. **Real-world relevance** - Widely used format
5. **Extension potential** - Translation, formatting, export features
6. **Educational value** - Demonstrates all interpreter phases

# Section 3: System Design

## Technology Stack

### Python Version
- **Python 3.13+** (latest features, improved type system)

### Built-in Libraries

- **`re`** - Regular expressions for pattern matching and tokenization
- **`time`** - Time simulation for real-time subtitle execution
- **`sys`** - System operations and exit codes
- **`pathlib`** - Modern file path operations
- **`typing`** - Type hints for code clarity and IDE support
- **`dataclasses`** - Immutable AST node structures with `frozen=True`
- **`hashlib`** - MD5 hashing for translation cache keys
- **`argparse`** - Command-line interface argument parsing

### Third-party Libraries

- **`deep-translator`** - Multi-language translation via Google Translate API
  - Supports 100+ languages
  - Free tier available
  - Used for Filipino, Korean, Chinese, Japanese translation

## Design Principles

### 1. Pipeline Architecture (Separation of Concerns)
```
Input → Lexer → Parser → Translator → Executor → Output
```
Each component has a single, well-defined responsibility.

### 2. Immutable AST Nodes
- Use `@dataclass(frozen=True)` for TimeStamp and SubtitleEntry
- Prevents accidental modification
- Thread-safe by design
- Easier debugging (no unexpected state changes)

### 3. Comprehensive Error Handling
- Custom exception hierarchy: `LexerError`, `ParserError`, `TranslationError`, etc.
- Descriptive error messages with context
- Early validation at each stage
- Graceful degradation (e.g., translation fallback to English)

### 4. Translation Caching Strategy
- File-based cache using MD5 hash of source content
- Cache key: `{md5_hash}_{source_lang}_{target_lang}.json`
- Instant retrieval for previously translated files
- Offline capability after first translation
- Batch translation to reduce API calls

### 5. Type Safety
- Full type hints throughout codebase
- IDE autocomplete support
- Early error detection
- Self-documenting code

In [ ]:
# Verify Python version and imports
import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print()

# Verify all required libraries
libraries = [
    're', 'time', 'pathlib', 'typing', 'dataclasses', 
    'hashlib', 'argparse', 'deep_translator'
]

print("Library availability:")
for lib in libraries:
    try:
        __import__(lib)
        print(f"  {lib:20} ✓ Available")
    except ImportError:
        print(f"  {lib:20} ✗ Missing")

# Section 4: Architecture Overview

## Data Flow Diagram

The following diagram illustrates the complete data flow through the interpreter pipeline:

```mermaid
flowchart TD
    A[Input .srt File] --> B[LEXER<br/>Tokenization<br/>Pattern Recognition]
    B -->|Token Stream<br/>INDEX, NEWLINE, TIMESTAMP, ARROW, ...| C[PARSER<br/>Syntax Analysis<br/>AST Construction]
    C -->|AST SubtitleEntry objects<br/>Entry1, Entry2, Entry3, ...| D[TRANSLATOR<br/>Semantic Transformation<br/>Multi-language, Optional]
    D -->|Translated AST| E{Output<br/>Selection}
    
    E -->|Execute| F[EXECUTOR]
    E -->|Analyze| G[STATS]
    E -->|Export| H[EXPORT]
    E -->|Format| I[FORMATTER]
    
    F --> J[Display]
    G --> K[Statistics]
    H --> L[Files]
    I --> M[ANSI Output]
```

## Component Descriptions

### 1. Lexer (`src/lexer.py`)

**Input:** Raw text string (file content)

**Output:** List of Token objects

**Responsibility:**
- Pattern recognition using regular expressions
- Character stream → token stream conversion
- Initial format validation

**Key Patterns:**
- Timestamp: `r'\d{2}:\d{2}:\d{2},\d{3}'`
- Arrow: `r'-->'`
- Index: `r'^\d+$'` (on its own line)

**Error Detection:**
- Invalid characters in timestamp
- Malformed token sequences

---

### 2. Parser (`src/parser.py`, `src/ast_nodes.py`)

**Input:** Token stream from Lexer

**Output:** List of SubtitleEntry AST nodes

**Responsibility:**
- Syntax validation (grammar enforcement)
- Semantic checks (time ordering, index sequence)
- AST construction with immutable nodes

**Validations Performed:**
1. Sequential indexes (1, 2, 3, ...)
2. Valid timestamp ranges (MM/SS ≤ 59, mmm ≤ 999)
3. Start time < end time
4. Non-empty text content
5. Proper block structure

**AST Nodes:**
- `TimeStamp`: Immutable time representation
- `SubtitleEntry`: Complete subtitle with validation

---

### 3. Translator (`src/translator.py`)

**Input:** AST + target language

**Output:** Translated AST (new SubtitleEntry objects with translated text)

**Responsibility:**
- Multi-language translation via Google Translate
- File-based caching for performance
- Batch processing to reduce API calls

**Supported Languages:**
- English (passthrough, no translation)
- Filipino (Tagalog)
- Korean
- Chinese (Simplified)
- Japanese

**Caching Strategy:**
- MD5 hash of source content as cache key
- JSON files stored in `.srt_cache/` directory
- Instant retrieval for repeated translations
- Fallback to English if translation fails

---

### 4. Executor (`src/executor.py`)

**Input:** AST + execution mode + formatting options

**Output:** Console display with timing

**Execution Modes:**

1. **Sequential**: Display each subtitle with brief pause (0.5s)
   - Fast demonstration mode
   - No timing simulation

2. **Real-time**: Display at actual timestamps
   - Faithful to original timing
   - Takes actual duration to complete

3. **Accelerated**: Display with speed multiplier
   - Configurable speed (e.g., 5x, 10x)
   - Maintains timing relationships

**Output Format:**
```
[HH:MM:SS.mmm] DISPLAY: "subtitle text"
[HH:MM:SS.mmm] CLEAR
```

---

### 5. Extensions

#### Statistics (`src/stats.py`)
- Total entries and duration
- Average subtitle duration and text length
- Longest/shortest by duration and text length

#### Export (`src/export.py`)
- **Text export**: Plain, numbered, or separated formats
- **SRT export**: Complete translated subtitle file

#### Formatter (`src/formatter.py`)
- HTML to ANSI escape code conversion
- Tag support: `<i>`, `<b>`, `<u>`, `<font color>`
- 24-bit RGB color support

## Error Handling Strategy

### Exception Hierarchy
```python
Exception
  ├── LexerError        # Tokenization failures
  ├── ParserError       # Syntax/semantic violations
  ├── TranslationError  # Translation failures
  ├── ExecutorError     # Execution failures
  ├── StatisticsError   # Statistics calculation errors
  ├── ExportError       # Export operation failures
  └── FormatterError    # Formatting conversion errors
```

### Error Messages
- Include specific context (line number, token, expected vs actual)
- Actionable suggestions when possible
- Clear indication of error location

### Graceful Degradation
- Translation fallback to English if API fails
- Continue execution after non-critical errors
- User-friendly error reporting

## Design Decisions & Justifications

### Why Pipeline Architecture?
- **Modularity**: Each component can be developed/tested independently
- **Maintainability**: Changes isolated to specific components
- **Extensibility**: Easy to add new features (e.g., new output formats)
- **Educational**: Clear demonstration of interpreter phases

### Why Frozen Dataclasses for AST?
- **Immutability**: Prevents accidental modifications
- **Thread safety**: Safe for concurrent operations
- **Debugging**: No unexpected state changes
- **Clarity**: Explicit about data flow

### Why File-based Caching?
- **Performance**: Instant retrieval for repeated translations
- **Offline capability**: Works without internet after first translation
- **Persistence**: Cache survives program restarts
- **Transparency**: User can inspect/clear cache files

### Why Batch Translation?
- **API efficiency**: Fewer API calls = faster execution
- **Rate limiting**: Reduces chance of hitting API limits
- **Atomicity**: All subtitles translated together
- **Consistency**: Same translation context for all entries

# Section 5: Implementation Details

This section demonstrates the core implementation of each component with code examples.

## 5.1 Lexer Implementation

In [ ]:
# Show Lexer token types and core logic
from src.lexer import Lexer, Token, TokenType
import inspect

print("="*60)
print("LEXER: Token Types")
print("="*60)
print("\nAvailable token types:")
for token_type in TokenType:
    print(f"  - {token_type.name}: {token_type.value}")

print("\n" + "="*60)
print("LEXER: Token Class Structure")
print("="*60)
print(inspect.getsource(Token))

In [ ]:
# Demonstrate lexer tokenization
sample_srt = """1
00:00:01,000 --> 00:00:03,000
Hello world!

2
00:00:04,000 --> 00:00:06,000
This is a <i>test</i>.

"""

print("="*60)
print("LEXER DEMONSTRATION")
print("="*60)
print("\nInput SRT content:")
print(sample_srt)
print("\nTokenization result:")
print("-"*60)

lexer = Lexer()
tokens = lexer.tokenize(sample_srt)

for i, token in enumerate(tokens, 1):
    print(f"{i:3}. {token}")

print("-"*60)
print(f"Total tokens generated: {len(tokens)}")

## 5.2 Parser Implementation

In [ ]:
# Show AST node structures
from src.ast_nodes import TimeStamp, SubtitleEntry

print("="*60)
print("AST NODE: TimeStamp")
print("="*60)
print(inspect.getsource(TimeStamp))

print("\n" + "="*60)
print("AST NODE: SubtitleEntry")
print("="*60)
print(inspect.getsource(SubtitleEntry))

In [ ]:
# Demonstrate parser with AST construction
from src.parser import Parser

print("="*60)
print("PARSER DEMONSTRATION")
print("="*60)
print("\nParsing the tokenized input...\n")

parser = Parser(tokens)
entries = parser.parse()

print(f"Parsed {len(entries)} subtitle entries:\n")

for entry in entries:
    print(f"Entry {entry.index}:")
    print(f"  Start time: {entry.start_time}")
    print(f"  End time:   {entry.end_time}")
    print(f"  Duration:   {(entry.end_time.to_milliseconds() - entry.start_time.to_milliseconds()) / 1000:.2f}s")
    print(f"  Text:       {entry.get_text()!r}")
    print(f"  Lines:      {entry.text}")
    print()

## 5.3 Translator Implementation

In [ ]:
# Demonstrate translation with caching
from src.translator import Translator

print("="*60)
print("TRANSLATOR DEMONSTRATION")
print("="*60)
print("\nOriginal entries (English):")
for entry in entries:
    print(f"  {entry.index}. {entry.get_text()}")

# Translate to Filipino
print("\nTranslating to Filipino...")
translator_fil = Translator('filipino')
translated_fil = translator_fil.translate_entries(entries, file_content=sample_srt)

print("\nTranslated entries (Filipino):")
for entry in translated_fil:
    print(f"  {entry.index}. {entry.get_text()}")

# Translate to Korean
print("\nTranslating to Korean...")
translator_ko = Translator('korean')
translated_ko = translator_ko.translate_entries(entries, file_content=sample_srt)

print("\nTranslated entries (Korean):")
for entry in translated_ko:
    print(f"  {entry.index}. {entry.get_text()}")

## 5.4 Executor Implementation

In [ ]:
# Demonstrate sequential execution mode
from src.executor import Executor

print("="*60)
print("EXECUTOR DEMONSTRATION: Sequential Mode")
print("="*60)
print()

executor = Executor()
executor.execute(entries, mode="sequential", enable_formatting=False)

print()
print("Execution complete!")

In [ ]:
# Demonstrate accelerated execution mode
print("="*60)
print("EXECUTOR DEMONSTRATION: Accelerated Mode (10x speed)")
print("="*60)
print()

executor_accel = Executor()
executor_accel.execute(entries, mode="accelerated", speed_factor=10.0, enable_formatting=False)

print()
print("Accelerated execution complete!")

## 5.5 Error Handling Implementation

In [ ]:
# Demonstrate error handling with invalid files
from src.lexer import LexerError
from src.parser import ParserError

invalid_test_cases = [
    ('examples/invalid_timestamp_order.srt', 'Start time after end time'),
    ('examples/invalid_missing_index.srt', 'Missing index'),
    ('examples/invalid_malformed_time.srt', 'Invalid timestamp format')
]

print("="*60)
print("ERROR HANDLING DEMONSTRATION")
print("="*60)

for filepath, expected_error in invalid_test_cases:
    print(f"\n{'-'*60}")
    print(f"Testing: {filepath}")
    print(f"Expected: {expected_error}")
    print(f"{'-'*60}")
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        
        print("File content:")
        print(content)
        
        lexer = Lexer()
        tokens = lexer.tokenize(content)
        
        parser = Parser(tokens)
        entries = parser.parse()
        
        print("UNEXPECTED: No error was raised!")
        
    except (LexerError, ParserError) as e:
        print(f"\n✓ Error caught successfully:")
        print(f"  Type: {type(e).__name__}")
        print(f"  Message: {e}")
    
    except Exception as e:
        print(f"\n✗ Unexpected error:")
        print(f"  Type: {type(e).__name__}")
        print(f"  Message: {e}")

# Section 6: Testing & Demonstration

This section runs comprehensive tests and demonstrates all features of the interpreter.

## 6.1 Testing Strategy

Our testing approach includes:

1. **Unit Tests** - Individual component testing
   - `test_lexer.py` - Tokenization tests
   - `test_parser.py` - Parsing and validation tests
   - `test_executor.py` - Execution mode tests
   - `test_translator.py` - Translation tests
   - `test_stats.py` - Statistics calculation tests
   - `test_export.py` - Export functionality tests
   - `test_formatter.py` - ANSI formatting tests

2. **Integration Tests** - Full pipeline testing
   - `test_integration.py` - End-to-end workflow tests

3. **Test Files** - Comprehensive examples
   - Valid: `valid_basic.srt`, `valid_multiline.srt`, `valid_formatting.srt`, `valid_complex.srt`
   - Invalid: `invalid_missing_index.srt`, `invalid_timestamp_order.srt`, etc.

**Test Coverage:**
- 29 test cases across 4 test files
- All components tested
- All execution modes validated
- All supported languages tested
- Error handling verified

In [ ]:
# Run basic file test
print("="*70)
print("TEST 1: Valid Basic File")
print("="*70)
!python main.py examples/valid_basic.srt --mode sequential

In [ ]:
# Run complex file with statistics
print("="*70)
print("TEST 2: Complex File with Statistics")
print("="*70)
!python main.py examples/valid_complex.srt --stats

In [ ]:
# Test multi-language translation
languages = ['filipino', 'korean', 'chinese', 'japanese']

for lang in languages:
    print("="*70)
    print(f"TEST 3.{languages.index(lang)+1}: Translation to {lang.title()}")
    print("="*70)
    !python main.py examples/valid_basic.srt --lang {lang} --mode sequential
    print()

In [ ]:
# Test ANSI formatting
print("="*70)
print("TEST 4: ANSI Formatting")
print("="*70)
!python main.py examples/valid_formatting.srt --format --mode sequential

In [ ]:
# Test text export (all formats)
print("="*70)
print("TEST 5: Text Export")
print("="*70)

formats = ['plain', 'numbered', 'separated']
for fmt in formats:
    output_file = f'demo_export_{fmt}.txt'
    print(f"\nExporting in {fmt} format to {output_file}...")
    !python main.py examples/valid_complex.srt --export-txt {output_file} --export-format {fmt}
    
    print(f"\nContent preview ({fmt} format):")
    with open(output_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:5]:  # Show first 5 lines
            print(f"  {line.rstrip()}")
    print(f"  ... ({len(lines)} total lines)")

In [ ]:
# Test SRT export
print("="*70)
print("TEST 6: SRT Export (Translated)")
print("="*70)
!python main.py examples/valid_basic.srt --export-srt demo_filipino.srt --lang filipino

print("\nExported SRT content:")
with open('demo_filipino.srt', 'r', encoding='utf-8') as f:
    print(f.read())

In [ ]:
# Test invalid files (error handling)
invalid_files = [
    'examples/invalid_missing_index.srt',
    'examples/invalid_timestamp_order.srt',
    'examples/invalid_malformed_time.srt',
    'examples/invalid_no_text.srt'
]

print("="*70)
print("TEST 7: Invalid Files (Error Handling)")
print("="*70)

for filepath in invalid_files:
    print(f"\n{'-'*70}")
    print(f"Testing: {filepath}")
    print(f"{'-'*70}")
    !python main.py {filepath} 2>&1

## Test Results Analysis

### Valid File Tests
- All valid files parse and execute correctly
- Translation works for all 5 supported languages
- ANSI formatting renders properly
- Export functions generate correct output files
- Statistics calculations are accurate

### Invalid File Tests
- All invalid files produce appropriate error messages
- Error messages are descriptive and include context
- Errors are caught at the correct stage (Lexer or Parser)
- No crashes or unexpected behavior

### Performance Observations
- Translation caching provides instant retrieval
- First translation takes ~2-3 seconds per language
- Cached translations are instantaneous
- Sequential mode is fastest for demonstration
- Accelerated mode maintains timing relationships

# Section 7: Extensions

Phase 5 extensions add significant functionality beyond the core interpreter.

## 7.1 Statistics Extension

In [ ]:
# Demonstrate statistics calculation
from src.stats import calculate_statistics

print("="*70)
print("STATISTICS EXTENSION")
print("="*70)

# Parse a file
with open('examples/valid_complex.srt', 'r', encoding='utf-8') as f:
    content = f.read()

lexer = Lexer()
tokens = lexer.tokenize(content)
parser = Parser(tokens)
entries = parser.parse()

print(f"\nAnalyzing {len(entries)} subtitle entries...\n")

# Calculate statistics
stats = calculate_statistics(entries)

# Display formatted statistics
print(stats.to_string())

print("\nKey Insights:")
print(f"  - Total runtime: {stats.format_duration(stats.total_duration_ms)}")
print(f"  - Average subtitle stays on screen for {stats.avg_duration_ms/1000:.2f} seconds")
print(f"  - Longest subtitle: Entry #{stats.longest_by_duration[0]} ({stats.longest_by_duration[1]/1000:.1f}s)")
print(f"  - Shortest subtitle: Entry #{stats.shortest_by_duration[0]} ({stats.shortest_by_duration[1]/1000:.1f}s)")
print(f"  - Most text: Entry #{stats.longest_by_text[0]} ({stats.longest_by_text[1]} characters)")
print(f"  - Least text: Entry #{stats.shortest_by_text[0]} ({stats.shortest_by_text[1]} characters)")

## 7.2 Export Extension

In [ ]:
# Demonstrate export functionality
from src.export import export_to_text, export_to_srt

print("="*70)
print("EXPORT EXTENSION")
print("="*70)

# Text export - Plain format
print("\n1. Plain Text Export")
export_to_text(entries, 'demo_plain.txt', format_type='plain')
with open('demo_plain.txt', 'r', encoding='utf-8') as f:
    content = f.read()
print("First 200 characters:")
print(content[:200] + "...")

# Text export - Numbered format
print("\n2. Numbered Text Export")
export_to_text(entries, 'demo_numbered.txt', format_type='numbered')
with open('demo_numbered.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
print("First 5 entries:")
for line in lines[:5]:
    print(f"  {line.rstrip()}")

# Text export - Separated format
print("\n3. Separated Text Export")
export_to_text(entries, 'demo_separated.txt', format_type='separated')
with open('demo_separated.txt', 'r', encoding='utf-8') as f:
    content = f.read()
print("First 300 characters (showing blank line separation):")
print(repr(content[:300]) + "...")

# SRT export with translation
print("\n4. SRT Export (Translated to Japanese)")
translator = Translator('japanese')
translated_jp = translator.translate_entries(entries[:3], file_content=content)  # First 3 entries
export_to_srt(translated_jp, 'demo_japanese.srt')

print("\nExported Japanese SRT:")
with open('demo_japanese.srt', 'r', encoding='utf-8') as f:
    print(f.read())

print("\nExport Summary:")
print("  - Plain text: Raw subtitle text only")
print("  - Numbered: Includes [index] prefix for each subtitle")
print("  - Separated: Blank lines between subtitles")
print("  - SRT export: Complete valid .srt file with timing")

## 7.3 ANSI Formatter Extension

In [ ]:
# Demonstrate ANSI formatting
from src.formatter import html_to_ansi, strip_html_tags, hex_to_ansi_color, ANSICode

print("="*70)
print("ANSI FORMATTER EXTENSION")
print("="*70)

# Test cases
test_cases = [
    ("<i>Italic text</i>", "Italic"),
    ("<b>Bold text</b>", "Bold"),
    ("<u>Underlined text</u>", "Underline"),
    ('<font color="#FF0000">Red text</font>', "Color (Red)"),
    ("<i><b>Italic and Bold</b></i>", "Nested tags"),
    ('<i>Italic</i> and <font color="#00FF00">Green</font>', "Multiple tags")
]

print("\nHTML to ANSI Conversion:")
print("-"*70)

for html_text, description in test_cases:
    ansi_text = html_to_ansi(html_text)
    stripped_text = strip_html_tags(html_text)
    
    print(f"\n{description}:")
    print(f"  HTML:     {html_text}")
    print(f"  ANSI:     {repr(ansi_text)}")
    print(f"  Stripped: {stripped_text}")
    print(f"  Rendered: {ansi_text}")  # Actual ANSI rendering

# Demonstrate hex color conversion
print("\n" + "-"*70)
print("Hex Color to ANSI Conversion:")
print("-"*70)

colors = [
    ("#FF0000", "Red"),
    ("#00FF00", "Green"),
    ("#0000FF", "Blue"),
    ("#FFFF00", "Yellow"),
    ("#FF00FF", "Magenta")
]

for hex_color, name in colors:
    ansi_code = hex_to_ansi_color(hex_color)
    print(f"\n{name} ({hex_color}):")
    print(f"  ANSI code: {repr(ansi_code)}")
    print(f"  Rendered:  {ansi_code}████{ANSICode.RESET} {name}")

print("\n" + "="*70)
print("ANSI Codes Reference:")
print("="*70)
print(f"  RESET:     {repr(ANSICode.RESET)}")
print(f"  BOLD:      {repr(ANSICode.BOLD)}")
print(f"  ITALIC:    {repr(ANSICode.ITALIC)}")
print(f"  UNDERLINE: {repr(ANSICode.UNDERLINE)}")
print(f"\nNote: \\x1b and \\033 represent the same ESC character")

# Section 8: Insights & Conclusions

## Lessons Learned

### 1. Importance of Separation of Concerns
The pipeline architecture proved invaluable:
- Each component could be developed and tested independently
- Bugs were isolated to specific stages
- New features (translation, formatting) integrated cleanly
- Code remained maintainable as complexity grew

### 2. Value of Comprehensive Error Handling
Investing in detailed error messages paid off:
- Users could quickly identify and fix issues
- Debugging was significantly easier
- Error context (line numbers, expected vs actual) was crucial
- Custom exception types enabled precise error handling

### 3. Benefits of Immutable AST Structures
Using frozen dataclasses for AST nodes:
- Prevented accidental modifications during translation/execution
- Made data flow explicit and traceable
- Eliminated entire classes of bugs
- Improved code clarity and maintainability

### 4. Translation Caching Impact
File-based caching dramatically improved performance:
- First translation: ~2-3 seconds
- Cached translation: Instant (< 0.01 seconds)
- Enabled offline operation after initial translation
- Simple JSON format made cache inspection easy

### 5. Type Hints Improve Maintainability
Comprehensive type hints throughout the codebase:
- Enabled excellent IDE autocomplete
- Caught type errors early
- Served as inline documentation
- Made refactoring safer

## Strengths

### 1. Clean Pipeline Architecture
- Clear separation between lexing, parsing, translation, and execution
- Each stage has well-defined inputs and outputs
- Easy to understand and extend

### 2. Comprehensive Validation
- Validation at each stage (lexer, parser, translator, executor)
- Multiple validation levels: syntax, semantics, timing
- Prevents invalid data from propagating

### 3. Multi-language Support
- 5 supported languages with intelligent caching
- Graceful degradation if translation fails
- Batch processing for efficiency

### 4. Multiple Execution Modes
- Sequential: Fast demonstration
- Real-time: Faithful to original timing
- Accelerated: Configurable speed
- Each mode useful for different scenarios

### 5. Extensible Design
- Easy to add new features (statistics, export, formatting)
- Extension modules integrate cleanly
- No modification of core components needed

## Limitations

### 1. Translation Dependency
- Requires internet connection for first translation
- Dependent on Google Translate API availability
- No support for offline-first translation models

### 2. Real-time Mode Constraint
- Real-time mode requires actual time to elapse
- Not suitable for long subtitle files (> 30 minutes)
- No skip/seek functionality

### 3. Limited HTML Tag Support
- Only basic formatting tags supported (<i>, <b>, <u>, <font>)
- No support for advanced SRT features:
  - Positioning tags ({\\an8})
  - Karaoke timing ({\\k})
  - Drawing commands

### 4. No Editing Capabilities
- Read-only interpretation
- Cannot modify subtitle timing or text
- Export creates new files rather than modifying existing

### 5. Terminal-based Output Only
- No graphical user interface
- ANSI formatting limited to terminal support
- No video overlay capability

## Future Improvements

### 1. Additional Subtitle Formats
- WebVTT (.vtt) support
- SubStation Alpha (.ssa/.ass) support
- Automatic format detection and conversion

### 2. Subtitle Editing Features
- Time shifting (adjust all timestamps)
- Merge/split subtitle entries
- Text find and replace
- Timing synchronization tools

### 3. Graphical User Interface
- Qt or Tkinter-based GUI
- Visual timeline editor
- Real-time preview with video
- Drag-and-drop file support

### 4. Advanced Positioning and Styling
- Full ASS/SSA tag support
- Custom font and color selection
- Subtitle positioning (top, bottom, custom)
- Animation effects

### 5. Offline Translation
- Integration with local translation models
- MarianMT or similar offline models
- Pre-downloaded language pairs
- Hybrid online/offline approach

### 6. Performance Optimization
- Streaming parser for large files
- Lazy evaluation of translations
- Parallel processing for batch operations
- Memory-mapped file support

### 7. Integration Features
- Video player plugins (VLC, MPV)
- Web service API (REST/GraphQL)
- Batch processing CLI tools
- Cloud storage integration

## Conclusion

This project successfully demonstrates the complete implementation of a language interpreter, from lexical analysis through parsing, translation, and execution. The SRT subtitle format proved to be an excellent choice for an educational interpreter project, offering clear structure for tokenization, rich validation opportunities, and real-world applicability.

The pipeline architecture, combined with comprehensive error handling and immutable data structures, resulted in a maintainable and extensible system. The addition of multi-language translation, statistics calculation, export functionality, and ANSI formatting showcases the flexibility of the core design.

Key takeaways:
- **Modular design** is crucial for complex systems
- **Error handling** should be prioritized from the start
- **Immutability** simplifies reasoning about program behavior
- **Caching** can dramatically improve performance
- **Type hints** enhance code quality and maintainability

This interpreter serves as both a functional tool for subtitle processing and a comprehensive demonstration of interpreter design principles applicable to any domain-specific language.

# Section 9: References

## Technical Documentation

1. **SubRip (.srt) Format Specification**
   - Matroska Subtitle Format Documentation
   - https://www.matroska.org/technical/subtitles.html

2. **Python Documentation**
   - Python 3.13 Official Documentation: https://docs.python.org/3/
   - Python `re` Module: https://docs.python.org/3/library/re.html
   - Python `dataclasses`: https://docs.python.org/3/library/dataclasses.html
   - Python Type Hints (PEP 484): https://peps.python.org/pep-0484/

3. **Third-party Libraries**
   - deep-translator: https://github.com/nidhaloff/deep-translator
   - deep-translator Documentation: https://deep-translator.readthedocs.io/

4. **ANSI Escape Codes**
   - ANSI Escape Code Reference: https://gist.github.com/fnky/458719343aabd01cfb17a3a4f7296797
   - Terminal Colors and Formatting: https://en.wikipedia.org/wiki/ANSI_escape_code

## Compiler and Interpreter Theory

5. **Compiler Design Principles**
   - Aho, A. V., Lam, M. S., Sethi, R., & Ullman, J. D. (2006). *Compilers: Principles, Techniques, and Tools* (2nd ed.). Addison-Wesley.
   - Concepts of lexical analysis, parsing, and AST construction

6. **Programming Language Implementation**
   - Grune, D., van Reeuwijk, K., Bal, H. E., Jacobs, C. J., & Langendoen, K. (2012). *Modern Compiler Design* (2nd ed.). Springer.
   - Error handling strategies and optimization techniques

## Course Materials

7. **CSS125L - Programming Languages**
   - Course lectures on interpreter design
   - Laboratory exercises on lexical analysis and parsing
   - Machine project specifications and requirements

## AI Assistance

8. **Claude Code (claude.ai/code)**
   - Used for code review and optimization suggestions
   - Assistance with test case generation
   - Documentation structure recommendations
   - Debugging complex translation caching logic

## Additional Resources

9. **Subtitle Processing**
   - Aegisub Advanced Subtitle Editor: https://github.com/Aegisub/Aegisub
   - pysubs2 Library (SRT/ASS parsing): https://github.com/tkarabela/pysubs2

10. **Translation APIs**
    - Google Cloud Translation API: https://cloud.google.com/translate/docs
    - Best practices for caching translation results

## Acknowledgments

- **Course Instructor**: For guidance on interpreter design principles and project requirements
- **CSS125L Teaching Team**: For comprehensive lectures on lexical analysis, parsing, and semantic analysis
- **Open Source Community**: For excellent libraries (deep-translator) that enabled multi-language support
- **Python Software Foundation**: For maintaining an excellent programming language and documentation
- **Classmates and Peers**: For testing the interpreter and providing feedback on usability

---

## Project Repository

**Project Structure:**
```
CSS125L_machine_project/
├── src/
│   ├── lexer.py          # Tokenization
│   ├── parser.py         # Syntax analysis
│   ├── ast_nodes.py      # AST structures
│   ├── translator.py     # Multi-language translation
│   ├── executor.py       # Subtitle execution
│   ├── interpreter.py    # Main orchestrator
│   ├── stats.py          # Statistics calculation
│   ├── export.py         # Export functionality
│   └── formatter.py      # ANSI formatting
├── tests/
│   ├── test_lexer.py
│   ├── test_parser.py
│   ├── test_translator.py
│   ├── test_executor.py
│   ├── test_stats.py
│   ├── test_export.py
│   ├── test_formatter.py
│   └── test_integration.py
├── examples/
│   ├── valid_basic.srt
│   ├── valid_multiline.srt
│   ├── valid_formatting.srt
│   ├── valid_complex.srt
│   └── invalid_*.srt (4 files)
├── main.py              # CLI entry point
├── demo.ipynb           # This notebook
└── README.md
```

**Total Lines of Code:** ~2,500 lines  
**Test Coverage:** 29 test cases  
**Supported Languages:** 5 (English, Filipino, Korean, Chinese, Japanese)  
**Documentation:** Complete with inline comments and docstrings

---

*This project was completed as part of CSS125L - Programming Languages course requirements.*

*All code is original work with assistance from AI tools for optimization and testing.*

---

# End of Demonstration

Thank you for exploring the SRT Subtitle Interpreter!

To run the interpreter from command line:
```bash
python main.py <file.srt> [options]

Options:
  --mode {sequential,real_time,accelerated}
  --speed SPEED_FACTOR
  --lang {english,filipino,korean,chinese,japanese}
  --stats
  --export-txt [PATH]
  --export-srt [PATH]
  --export-format {plain,numbered,separated}
  --format
```

For more information, run: `python main.py --help`